<a href="https://colab.research.google.com/github/IrisGun/Decision-Tree-Predicting-survivals-from-the-Titanic-Disaster/blob/main/Titanic_survival_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem 2 - Decision Tree: Predicting survivals from the Titanic Disaster (50 points)




## Xây dựng mô hình dự đoán người sống sốt sau thảm họa tàu Titanic.

https://query.data.world/s/ckuvwmg55m3tkazkrw3u2os35jm6cg

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score, plot_confusion_matrix
from sklearn.tree import DecisionTreeClassifier



# plotting
import matplotlib
import matplotlib.pyplot as plt
font = {'size' : 18}
matplotlib.rc('font', **font)

pd.set_option('display.max_columns',None)

## 1. Data loading (10 points)
Requirement:
  - Load the dataset from csv to pandas. (5 points)

In [2]:
## YOUR CODE HERE ##
data='https://query.data.world/s/ckuvwmg55m3tkazkrw3u2os35jm6cg'
df=pd.read_csv(data)
print(df.shape)
df.head(n=20)
df.tail()

(1310, 14)


,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
1305,3.0,0.0,"Zabour, Miss. Thamine",female,NaN,1.0,0.0,2665,14.4542,NaN,C,NaN,NaN,NaN
1306,3.0,0.0,"Zakarian, Mr. Mapriededer",male,26.5,0.0,0.0,2656,7.2250,NaN,C,NaN,304.0,NaN
1307,3.0,0.0,"Zakarian, Mr. Ortin",male,27.0,0.0,0.0,2670,7.2250,NaN,C,NaN,NaN,NaN
1308,3.0,0.0,"Zimmerman, Mr. Leo",male,29.0,0.0,0.0,315082,7.8750,NaN,S,NaN,NaN,NaN
1309,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


First glance and Summary:\
__Feature:__


1.   survival: 0 - No, 1 - Yes
2.   calss: Passenger Class (1st, 2nd, 3rd)
3. sibsp: Number of Siblings/Spouses Abroard
4. parch - Number of Parents/Children Aboard
5. ticket: Ticket number
6. fare: Passenger Fare
7. embarked: Port of Embarkation (C=Cherbourg, Q=Queenstown, S=Southampton)
8. boat: Lifeboat (if survived)
9. body: Body number (if did not survive and body was recovered)
10. home_dest: Home destination (?)
11. name: Name of passensger which includes their title after the comma regarding gender and marital status such as Mr., Ms., .. job and position such as Col. (colonel), Dr., etc.



In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1310 entries, 0 to 1309
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   pclass     1309 non-null   float64
 1   survived   1309 non-null   float64
 2   name       1309 non-null   object 
 3   sex        1309 non-null   object 
 4   age        1046 non-null   float64
 5   sibsp      1309 non-null   float64
 6   parch      1309 non-null   float64
 7   ticket     1309 non-null   object 
 8   fare       1308 non-null   float64
 9   cabin      295 non-null    object 
 10  embarked   1307 non-null   object 
 11  boat       486 non-null    object 
 12  body       121 non-null    float64
 13  home.dest  745 non-null    object 
dtypes: float64(7), object(7)
memory usage: 143.4+ KB


  - Break down the dataset into X (features) and y(label) ~ to be predicted. (5 points)\
  Chia dữ liệu thành X (features) và y (label) cần dự đoán.

In [4]:
#The last row only contains index => drop
df=df[:-1]
df.isnull().sum()/len(df)*100

pclass        0.000000
survived      0.000000
name          0.000000
sex           0.000000
age          20.091673
sibsp         0.000000
parch         0.000000
ticket        0.000000
fare          0.076394
cabin        77.463713
embarked      0.152788
boat         62.872422
body         90.756303
home.dest    43.086325
dtype: float64

In [5]:
df.describe()

,pclass,survived,age,sibsp,parch,fare,body
count,1309.000000,1309.000000,1046.000000,1309.000000,1309.000000,1308.000000,121.000000
mean,2.294882,0.381971,29.881135,0.498854,0.385027,33.295479,160.809917
std,0.837836,0.486055,14.413500,1.041658,0.865560,51.758668,97.696922
min,1.000000,0.000000,0.166700,0.000000,0.000000,0.000000,1.000000
25%,2.000000,0.000000,21.000000,0.000000,0.000000,7.895800,72.000000
50%,3.000000,0.000000,28.000000,0.000000,0.000000,14.454200,155.000000
75%,3.000000,1.000000,39.000000,1.000000,0.000000,31.275000,256.000000
max,3.000000,1.000000,80.000000,8.000000,9.000000,512.329200,328.000000


In [6]:
## YOUR CODE HERE ##
X = df.drop(columns=['survived','boat'])
y = df.survived

## 2. Data preprocessing (15 points)

Requirement:
  - Convert the text data into numerical data.\
  Chuyển dữ liệu dạng chữ thành dạng số.

In [7]:
## YOUR CODE HERE ##
df.info()
for i in df.select_dtypes(include='object'):
  print(i, df[i].nunique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   pclass     1309 non-null   float64
 1   survived   1309 non-null   float64
 2   name       1309 non-null   object 
 3   sex        1309 non-null   object 
 4   age        1046 non-null   float64
 5   sibsp      1309 non-null   float64
 6   parch      1309 non-null   float64
 7   ticket     1309 non-null   object 
 8   fare       1308 non-null   float64
 9   cabin      295 non-null    object 
 10  embarked   1307 non-null   object 
 11  boat       486 non-null    object 
 12  body       121 non-null    float64
 13  home.dest  745 non-null    object 
dtypes: float64(7), object(7)
memory usage: 143.3+ KB
name 1307
sex 2
ticket 929
cabin 186
embarked 3
boat 27
home.dest 369


_Idea_ \
1. _name_ \
Unique name seems not useful at all => Extract only title and see if it has any contribution as the title may reflect passenger's social status and it deems to effect their chance of survive.\
Obtain the title by splitting name to take only string after the comma (,) and before the period (.)

2. _cabin_ \
Cabin may have slight effect on the survival chance of the passenger. For example the nearer the cabin is to the stairs, the higher chance they may survival. \
But as no such information can be seen from the dataset and missing rate is fairly high => DROP this columns. \
Same thing happens to *'ticket', 'body', and 'home.dest'* column as they show no rational contribution.



In [8]:
#Obtain title column:
df['title']=df['name'].str.split('.').str[0]
df['title']=df['title'].str.split(', ').str[1]
df.head()


#Drop columns
df.drop(columns=['name','ticket','cabin','body','home.dest', 'boat'], axis=1, inplace=True)
df.head()

,pclass,survived,sex,age,sibsp,parch,fare,embarked,title
0,1.0,1.0,female,29.0000,0.0,0.0,211.3375,S,Miss
1,1.0,1.0,male,0.9167,1.0,2.0,151.5500,S,Master
2,1.0,0.0,female,2.0000,1.0,2.0,151.5500,S,Miss
3,1.0,0.0,male,30.0000,1.0,2.0,151.5500,S,Mr
4,1.0,0.0,female,25.0000,1.0,2.0,151.5500,S,Mrs


In [9]:
df.info()
for i in df.select_dtypes(include=['object']):
  print(i, df[i].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   pclass    1309 non-null   float64
 1   survived  1309 non-null   float64
 2   sex       1309 non-null   object 
 3   age       1046 non-null   float64
 4   sibsp     1309 non-null   float64
 5   parch     1309 non-null   float64
 6   fare      1308 non-null   float64
 7   embarked  1307 non-null   object 
 8   title     1309 non-null   object 
dtypes: float64(6), object(3)
memory usage: 92.2+ KB
sex male      843
female    466
Name: sex, dtype: int64
embarked S    914
C    270
Q    123
Name: embarked, dtype: int64
title Mr              757
Miss            260
Mrs             197
Master           61
Rev               8
Dr                8
Col               4
Mlle              2
Ms                2
Major             2
Capt              1
Sir               1
Dona              1
Jonkheer          1
the C

#### Ideas for encoding:


2. __embarked__ \
Encoded by pd.get_dummies

3. __title__ \
* There are some titles that specify passengers' high social status like: the Countess, Jonkheer, Sir, etc. => Group them into one category called "Noble"
* Some categories seem mispelling or inconsistent: Mlle, Mme \
_Note: the catergorization is relatively acceptable only as I have limited knowledge in this field. \
(Oh gosh it was like a Western history class now_  ㅠㅠ)
* As per above assumption, __title__ should be encoded as an ordinal catergorical variable. 
4. __pclass__\
has d_type as float but it's actually a ordinal catergorical variable => Let it be

In [10]:
#ENCODING: 3 columns that need to be encoded: sex, embarked, title.

#Encoding gender
df['sex']=df.sex.apply(lambda x:0 if x=='female' else 1)


In [11]:
#Encode title

#Show number of survivals/total number of people grouped by their title.
df.groupby(['title']).mean().sort_values(['survived'], ascending=False)['survived']

title
the Countess    1.000000
Mlle            1.000000
Sir             1.000000
Dona            1.000000
Mme             1.000000
Lady            1.000000
Mrs             0.786802
Miss            0.676923
Master          0.508197
Col             0.500000
Major           0.500000
Dr              0.500000
Ms              0.500000
Mr              0.162483
Jonkheer        0.000000
Rev             0.000000
Don             0.000000
Capt            0.000000
Name: survived, dtype: float64

From the above mean (number of survivors divided by total number of people belong to the title group) the Analysist assume the survival chance related to title as: \
Man: 1 \
Master: 2 \
Miss: 3 \
Mrs: 4 \
Noble: 5

In [12]:
df['title_']=df['title']
df['title_']=df['title_'].replace(['the Countess', 'Dona', 'Lady', 'Col', 'Capt', 'Dr', 'Jonkheer',
                                 'Rev', 'Don', 'Sir', 'Major'], 'Noble')
df['title_'] = df['title_'].replace({'Mlle': 'Miss', 'Ms': 'Miss', 'Mme': 'Mrs'})
df.title_.value_counts()
#df.groupby(['title_']).mean().sort_values(['survived'], ascending=False)['survived']


Mr        757
Miss      264
Mrs       198
Master     61
Noble      29
Name: title_, dtype: int64

In [13]:
df['title_'] = df.title_.map({'Mr':1, 'Master':2, 'Miss':3, 'Mrs': 4, 'Noble': 5})
df.head()

,pclass,survived,sex,age,sibsp,parch,fare,embarked,title,title_
0,1.0,1.0,0,29.0000,0.0,0.0,211.3375,S,Miss,3
1,1.0,1.0,1,0.9167,1.0,2.0,151.5500,S,Master,2
2,1.0,0.0,0,2.0000,1.0,2.0,151.5500,S,Miss,3
3,1.0,0.0,1,30.0000,1.0,2.0,151.5500,S,Mr,1
4,1.0,0.0,0,25.0000,1.0,2.0,151.5500,S,Mrs,4


In [14]:
#Encode embarked:

df_2=pd.get_dummies(df, columns=['embarked'], drop_first= True)
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   pclass      1309 non-null   float64
 1   survived    1309 non-null   float64
 2   sex         1309 non-null   int64  
 3   age         1046 non-null   float64
 4   sibsp       1309 non-null   float64
 5   parch       1309 non-null   float64
 6   fare        1308 non-null   float64
 7   title       1309 non-null   object 
 8   title_      1309 non-null   int64  
 9   embarked_Q  1309 non-null   uint8  
 10  embarked_S  1309 non-null   uint8  
dtypes: float64(6), int64(2), object(1), uint8(2)
memory usage: 94.7+ KB


In [15]:
#Impute numeric Nan value with mean;
for i in df_2[['age','fare']]:
  df_2[i].fillna(df_2[i].mean(), inplace=True)
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   pclass      1309 non-null   float64
 1   survived    1309 non-null   float64
 2   sex         1309 non-null   int64  
 3   age         1309 non-null   float64
 4   sibsp       1309 non-null   float64
 5   parch       1309 non-null   float64
 6   fare        1309 non-null   float64
 7   title       1309 non-null   object 
 8   title_      1309 non-null   int64  
 9   embarked_Q  1309 non-null   uint8  
 10  embarked_S  1309 non-null   uint8  
dtypes: float64(6), int64(2), object(1), uint8(2)
memory usage: 94.7+ KB


## 3. Train (25 points)

Requirement:
- Split the dataset into train & test sets. (5 points)\
Chia dữ liệu thành 2 tập train và test.

In [16]:
## YOUR CODE HERE ##
test_size_ratio = 0.3
X_train, X_test, y_train, y_test = train_test_split(df_2.drop(columns=['survived', 'title']), df_2.survived,
                                                    test_size=test_size_ratio, random_state=1)

Requirement:
- Train a Decision tree classifier on the train set. (20 points)\
Dùng decision tree để học dữ liệu train.

In [17]:
## YOUR CODE HERE ##
DT_classifier = DecisionTreeClassifier().fit(X_train, y_train)

## 4. Predict & Test (30 points)
Requirement:
- Use the trained decision tree classifier to make predictions on the test set. (15 points)\
Dùng decision tree để dự đoán dữ liệu test.

In [18]:
## YOUR CODE HERE ##
y_pred = DT_classifier.predict(X_test)


Requirement:
- Evaluate the predictions.\
Đo lường kết quả của mô hình. (15 points)

In [19]:
## YOUR CODE HERE ##
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score

print(accuracy_score(y_test, y_pred))
print(precision_score(y_test, y_pred))
print(recall_score(y_test, y_pred))
print(f1_score(y_test, y_pred))

0.732824427480916
0.6666666666666666
0.6624203821656051
0.6645367412140575


## 5. (Advanced topics) Analysis (20 points)
Requirement:
- Adjust some hyperparameters of the decision tree classifier (max_depth, min_sample_split) to improve the performance further. (10 points)\
Thử thay đổi các yếu tố trong decision trê để tìm ra mô hình tối ưu (max_depth, min_sample_split).


In [20]:
## YOUR CODE HERE ##
from sklearn.model_selection import GridSearchCV 

params ={'max_depth': list(range(2,50)) , 'min_samples_split':[2,3,4,5]}

clf = GridSearchCV(estimator = DecisionTreeClassifier(random_state=1),
             param_grid = params, verbose=4)

clf.fit(X_train, y_train)
print(clf.best_params_)

Fitting 5 folds for each of 192 candidates, totalling 960 fits
[CV 1/5] END ..max_depth=2, min_samples_split=2;, score=0.788 total time=   0.0s
[CV 2/5] END ..max_depth=2, min_samples_split=2;, score=0.814 total time=   0.0s
[CV 3/5] END ..max_depth=2, min_samples_split=2;, score=0.792 total time=   0.0s
[CV 4/5] END ..max_depth=2, min_samples_split=2;, score=0.792 total time=   0.0s
[CV 5/5] END ..max_depth=2, min_samples_split=2;, score=0.754 total time=   0.0s
[CV 1/5] END ..max_depth=2, min_samples_split=3;, score=0.788 total time=   0.0s
[CV 2/5] END ..max_depth=2, min_samples_split=3;, score=0.814 total time=   0.0s
[CV 3/5] END ..max_depth=2, min_samples_split=3;, score=0.792 total time=   0.0s
[CV 4/5] END ..max_depth=2, min_samples_split=3;, score=0.792 total time=   0.0s
[CV 5/5] END ..max_depth=2, min_samples_split=3;, score=0.754 total time=   0.0s
[CV 1/5] END ..max_depth=2, min_samples_split=4;, score=0.788 total time=   0.0s
[CV 2/5] END ..max_depth=2, min_samples_split=

In [21]:
clf.best_score_

0.8067533856022809

In [22]:
model_2=DecisionTreeClassifier(max_depth=8, min_samples_split=2).fit(X_train, y_train) 
y_pred = model_2.predict(X_test) 
print(accuracy_score(y_test, y_pred)) 
print(precision_score(y_test, y_pred)) 
print(recall_score(y_test, y_pred)) 
print(f1_score(y_test, y_pred))

0.7430025445292621
0.6772151898734177
0.6815286624203821
0.6793650793650793


In [23]:
model_2=DecisionTreeClassifier(max_depth=5, min_samples_split=2).fit(X_train, y_train) 
y_pred = model_2.predict(X_test) 
print(accuracy_score(y_test, y_pred)) 
print(precision_score(y_test, y_pred)) 
print(recall_score(y_test, y_pred)) 
print(f1_score(y_test, y_pred))

0.8193384223918575
0.7792207792207793
0.7643312101910829
0.7717041800643087


**Conclution:** From the above result => Select max_depth = 5, min_samples_split = 2 as this combination produce highest score with lowest computational cost.


###__???QUESTION???__
```
 clf.best_params_
```
cho kết quả max_depth =8 và min_samples_split = 2 là bộ parameters tốt nhất.
Nhưng score của bộ max_depth = 5, min_samples_split =2 cao nhất, đồng thời cho kết quả evaluate mô hình tốt nhât. \
Tại sao lại có hiện tượng này?

Requirement:
- Estimate the feature importance and identify the features that are the most influential to the prediction of whether a passenger survives. (5 points)\
Tính feature importance và tìm ra các features nào ảnh hưởng lớn nhất đến việc dự đoán xem một hành khách có sống sót hay không.

In [24]:
## YOUR CODE HERE ##

importance = DT_classifier.feature_importances_
importance_df = pd.DataFrame(list(zip(X_train.columns,importance)), 
                             columns = ['Feauture', 'Importance Score'])
importance_df.sort_values(['Importance Score'], ascending = False)

,Feauture,Importance Score
6,title_,0.329768
5,fare,0.268894
2,age,0.219648
0,pclass,0.108843
8,embarked_S,0.023778
3,sibsp,0.021080
4,parch,0.018906
7,embarked_Q,0.007958
1,sex,0.001125


Requirement:
- Are there any patterns in the group of passengers who have a high rate of survival? Why do you think it is the case? (5 points)\
Đưa ra nhận xét của bạn về patterns của nhóm sống sót cao và theo b ạn nghĩ tại sao lại như vậy.

In [25]:
import plotly.express as px
fig = px.scatter_3d(df.survived, x=df.title_, y=df.fare, z=df.age,
                    color = df.survived,
                    labels = {'x': 'Title', 'y': 'Fare', 'z': 'Age'})
fig.show()

Từ graph có thể sơ bộ thấy nhóm hành khách nữ mua vé có giá >100, tuổi từ 20-60 thuộc nhóm có tỉ lệ sống sót cao

Giải thích: 
- Phụ nữ và trẻ em thường được ưu tiên trong các thảm hoạ
- Nhóm 20-60 có sức khoẻ dẻo dai hơn độ tuổi quá nhỏ hoặc quá cao
- Giá vé >100. Có thể do hạng vé rẻ thường bị xếp vào các khoang dưới cùng gần đáy thuyền, khó thoát hiểm hơn khi xảy ra chìm tàu

###Hạn chế và phương án tiếp tục nghiên cứu:
Do thời gian giới hạn và kiến thức hạn chế nên kết quả cho ra bộ evaluation metrics chưa tối ưu. Các cách có thể thử để tạo ra mô hình tốt hơn:
- Scale biến fare
- Encode biến pclass thành dạng dummy 
- Impute age bằng kmeans 
- Chia lại nhóm title. Zip title và gender tạo thành 1 biến mới
- Thêm biến category của cabin 
- Chỉ chọn top k nhật định features có features importance cao nhất để đưa vào mô hình
- et cetera.